# Preprocess the scraped data

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import re
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

def init_spark(app_name: str):
    spark = SparkSession.builder.appName(app_name).getOrCreate()
    sc = spark.sparkContext
    return spark, sc

spark, sc = init_spark('Proj')
reg = re.compile(",") 

In [2]:
proj_dir = os.path.dirname(os.path.dirname(os.getcwd()))

# read file into spark df
open_position_data = spark.read.csv(f'{proj_dir}\data\scrapedData\open_positions_data.csv', header=True)

In [3]:
# Create a new column indicating if the row had a "+" character or not
cleaned_open_position_data = open_position_data.withColumn("has_plus", col("jobs count").contains("+"))

# Remove "+" character and commas from "jobs count" column and cast to integer
cleaned_open_position_data = cleaned_open_position_data.withColumn(
    "jobs_count_clean", 
    regexp_replace(regexp_replace(col("jobs count"), "\\+", ""), ",", "").cast("integer")
)
# Show the updated DataFrame
# cleaned_open_position_data.show()



In [4]:
# Convert jobs_count_clean column to integer and replace nulls with a default value (e.g., 0)
cleaned_open_position_data = cleaned_open_position_data.withColumn(
    "jobs_count_clean_int", 
    when(col("jobs_count_clean").cast("int").isNull(), 0).otherwise(col("jobs_count_clean").cast("int"))
)

cleaned_open_position_data_ordered = cleaned_open_position_data.orderBy("jobs_count_clean_int", ascending=False)
# cleaned_open_position_data_ordered.show(5)


In [5]:
cleaned_open_position_data_ordered = cleaned_open_position_data_ordered.dropDuplicates(subset=["company"]).orderBy("jobs_count_clean_int", ascending=False)
# cleaned_open_position_data_ordered.show(5)

In [6]:
from pyspark.sql.functions import dense_rank
from pyspark.sql.window import Window

# Define a window specification ordered by the column you want to rank by
window_spec = Window.orderBy(desc("jobs_count_clean_int"))

# Add a new column with dense rank
cleaned_open_position_data_ordered = cleaned_open_position_data_ordered.withColumn("work_opportunities_rank", dense_rank().over(window_spec))

# cleaned_open_position_data_ordered.show(20)

# Analysis

In [7]:
analysis_df = cleaned_open_position_data_ordered.select("company", "jobs_count_clean_int", "work_opportunities_rank")

In [8]:
analysis_df

AttributeError: 'DataFrame' object has no attribute 'display'

## Find top known companies rank

In [ ]:
gampa_rows = analysis_df.filter((analysis_df.company == "Microsoft")| (analysis_df.company == "Amazon") | (analysis_df.company == "Google") | (analysis_df.company == "Meta") | (analysis_df.company == "Apple inc.") | (analysis_df.company == "Netflix") | (analysis_df.company == "Tesla") | (analysis_df.company == "Uber") | (analysis_df.company == "Lyft") | (analysis_df.company == "Airbnb") | (analysis_df.company == "Slack") | (analysis_df.company == "Pinterest") | (analysis_df.company == "Snapchat") | (analysis_df.company == "Twitter") | (analysis_df.company == "Dropbox"))
gampa_rows

## Distribution for companies per location

In [ ]:
companies_per_location = analysis_df.groupBy("work_opportunities_rank").count()
companies_per_location

In [ ]:
# Cast columns to int
companies_per_location = companies_per_location.withColumn("work_opportunities_rank", col("work_opportunities_rank").cast("int"))
companies_per_location = companies_per_location.withColumn("count", col("count").cast("int"))

# Collect column values into lists of Row objects
companies_per_location_rank_rows = companies_per_location.select("work_opportunities_rank").collect()
companies_per_location_count_rows = companies_per_location.select("count").collect()

# Extract values from Row objects
companies_per_location_rank = [row['work_opportunities_rank'] for row in companies_per_location_rank_rows]
companies_per_location_count = [row['count'] for row in companies_per_location_count_rows]

# Expand 'work_opportunities_rank' according to 'count'
expanded_rank = np.repeat(companies_per_location_rank, companies_per_location_count)

# Calculate statistics
median_rank = np.median(expanded_rank)
mean_rank = np.mean(expanded_rank)
q75_rank, q25_rank = np.percentile(expanded_rank, [75 ,25])
iqr_rank = q75_rank - q25_rank

# Plot
bars = plt.bar(companies_per_location_rank, companies_per_location_count)

# Add labels and title
plt.xlabel('Work Opportunities Rank')
plt.ylabel('Count')
plt.title('Companies per Rank')

# Add statistics as lines
plt.axvline(median_rank, color='r', linestyle='dashed', linewidth=2, label=f'Median: {median_rank}')
plt.axvline(mean_rank, color='g', linestyle='dashed', linewidth=2, label=f'Mean: {mean_rank}')
plt.axvspan(q25_rank, q75_rank, color='b', alpha=0.1, label=f'IQR: {iqr_rank}')

# Create custom legend
legend_elements = [Line2D([0], [0], color='b', lw=1, label='oppurtunities per rank'),
                   Line2D([0], [0], color='r', linestyle='dashed', lw=2, label=f'Median: {median_rank}'),
                   Line2D([0], [0], color='g', linestyle='dashed', lw=2, label=f'Mean: {mean_rank}'),
                   Line2D([0], [0], color='b', alpha=0.1, lw=10, label=f'IQR: {iqr_rank}')]

# Add legend
plt.legend(handles=legend_elements)

plt.show()

## Find top 0.5% companies in work opportunities

In [ ]:
# Find top 0.25% companies in work opportunities
top_025_percent_companies = analysis_df.filter(analysis_df.work_opportunities_rank <= 0.0025 * analysis_df.count())
print(f"Top 0.25% companies in work opportunities: {top_025_percent_companies.count()} out of total {analysis_df.count()} companies")
top_025_percent_companies


## Bubble plot - top 10 companies

In [ ]:
# # Select top 10 companies
# top_companies = analysis_df.orderBy(col("jobs_count_clean_int").desc()).limit(10)

# # Collect column values into lists of Row objects
# top_companies_company_rows = top_companies.select("company").collect()
# top_companies_rank_rows = top_companies.select("work_opportunities_rank").collect()
# top_companies_count_rows = top_companies.select("jobs_count_clean_int").collect()

# # Extract values from Row objects
# top_companies_company = [row['company'] for row in top_companies_company_rows]
# top_companies_rank = [row['work_opportunities_rank'] for row in top_companies_rank_rows]
# top_companies_count = [row['jobs_count_clean_int'] for row in top_companies_count_rows]



# # Create bubble plot
# plt.scatter(top_companies_rank, top_companies_count, s=100)

# # Add labels and title
# plt.xlabel('Work Opportunities Rank')
# plt.ylabel('Jobs Count')
# plt.title('Top 10 Companies')

# # Add company names as annotations
# for i in range(len(top_companies_company)):
#     plt.annotate(top_companies_company[i], (top_companies_rank[i], top_companies_count[i]))

# plt.show()